## Import Libraries

In [4]:
from matplotlib import pyplot as plt
import torch
import numpy as np
import random
from Util.Transformers import Conv2dFilter
from Util.Pooling import MeanPooling
from matplotlib.gridspec import GridSpec
from Coding.Encoders import TTFSCoding, PoissonCoding
from Util.Learning import Conv2dSTDP
from Util.Dendrites import Conv2dDendriticInput, AveragePool2D
from Util.Filters import DoGFilter, GaborFilter
from Coding.Encoders import PoissonCoding
from pymonntorch import Network, NeuronGroup, Recorder, EventRecorder, SynapseGroup
from Util.Synapse import DDFSynapse
from Util.Current import ConstantCurrent
from Util.TimeResolution import TimeResolution
from Models.LIF import LIF
from matplotlib import pyplot as plt
import torch
from matplotlib import pyplot as plt
import numpy as np
from Util.Learning import SimpleSTDP
from Util.Axon import NeuronAxon
from Util.Specs import SpikeTrace
from Util.Input import InputBehavior

## Set Seeds

In [3]:
seed = 2020
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
net = Network(behavior={1: TimeResolution(dt=0.01)},
              dtype=torch.float64, tag='NET')

neuron_model = LIF(
    tau=10,
    u_rest=-65,
    u_reset=-70,
    threshold=-40,
    R=10,
    v_init_mode="normal(-40, 10)",
)

filtered_image_on_center = Conv2dFilter(filters=DoGFilter(10, 1, 1.5, one_sum=True,
                                                          zero_mean=True))((torch.from_numpy(
                                                              './data/bird.tif').unsqueeze(0).to(torch.float32))).squeeze(0)

input_ayer = NeuronGroup(
    size=256 * 256,
    net=net,
    behavior={
        2: InputBehavior(spikes=filtered_image_on_center.flatten() / 256),
        5: SpikeTrace(tau_s=10),
        6: NeuronAxon(have_trace=True),
        10: EventRecorder("spike", tag="ng_evrec"),
    },
    tag='input_layer'
)